<a href="https://colab.research.google.com/github/AsraniSanjana/All_Codes/blob/main/All_Semester_Codes/NLP_sem7/ColabFiles/NLP04_N_Gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# D17B_01_Sanjana Asrani
NLP
Lab 04 : N gram Language Model

In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Predicting Probability of a sentence

Considering bi-grams and uni-grams. similar implementation for trigram can also be done.

In [ ]:

'''
query_sentence = ["<s>", "i", "i", "am", "not", "</s>"]
'''
'''
corpus_text = [
    "<s> i am a human </s>",
    "<s> i am not a store </s>",
    "<s> i i live in mumbai </s>"
]
'''
print()

# Extracting n-grams

In [ ]:
import re
def n_grams(text,n):
	tokens=re.split("\\s+",text)
	ngrams= [ ]
	for i in range (len(tokens)-n+1):
		temp=[tokens[j] for j in range(i, i+n)]
		ngrams.append(" ".join(temp))
	return ngrams
text="the quick brown fox jumps over the lazy dog";
n_grams(text,3)       # trigram

['the quick brown',
 'quick brown fox',
 'brown fox jumps',
 'fox jumps over',
 'jumps over the',
 'over the lazy',
 'the lazy dog']

In [ ]:
'''
    corpus_text = [
        "<s> i am a human </s>",
        "<s> i am not a store </s>",
        "<s> i i live in mumbai </s>"
    ]
'''

'\n    corpus_text = [\n        "<s> i am a human </s>",\n        "<s> i am not a store </s>",\n        "<s> i i live in mumbai </s>"\n    ]\n'

In [ ]:
import re
import math
import random
from nltk.probability import FreqDist, ConditionalFreqDist
from nltk import bigrams, word_tokenize
from tabulate import tabulate

def calculate_counts(text_data):
    unigrams = []
    bigrams = []
    unigram_counts = {}
    bigram_counts = {}
    word_count = 0

    for sentence in text_data:
        for i in range(len(sentence) - 1):
            unigrams.append(sentence[i])
            bigrams.append((sentence[i], sentence[i + 1]))

            if (sentence[i], sentence[i + 1]) in bigram_counts:
                bigram_counts[(sentence[i], sentence[i + 1])] += 1
            else:
                bigram_counts[(sentence[i], sentence[i + 1])] = 1

        unigrams.append(sentence[i + 1])

        for i in range(len(sentence)):
            if sentence[i] in unigram_counts:
                unigram_counts[sentence[i]] += 1
            else:
                unigram_counts[sentence[i]] = 1

            word_count += 1

    return unigrams, bigrams, bigram_counts, unigram_counts, word_count

def calculate_unigram_probabilities(unigrams, unigram_counts, word_count):
    probabilities = {}
    for unigram in unigrams:
        probabilities[unigram] = float(unigram_counts[unigram] / word_count)
    return probabilities

def calculate_bigram_probabilities(bigrams, bigram_counts, unigram_counts, unigram_probabilities, smooth):
    probabilities = {}
    for bigram in bigrams:
        word_1 = bigram[0]

        if bigram in bigram_counts and word_1 in unigram_counts:
            probabilities[bigram] = float(bigram_counts[bigram] / unigram_counts[word_1])
        else:
            probabilities[bigram] = 0

        if smooth:
            if word_1 in unigram_counts:
                probabilities[bigram] = (0.5 * probabilities[bigram]) + (0.5 * unigram_probabilities[word_1])
            else:
                probabilities[bigram] = 0.5 * probabilities[bigram]

    return probabilities

def calculate_sentence_probability(sentence, unigram_counts, bigram_counts, unigram_probabilities, word_count, smooth, use_log):
    sentence_probability = 1
    if use_log:
        sentence_probability = 0

    for i in range(len(sentence) - 1):
        bigram_probability = calculate_bigram_probabilities([(sentence[i], sentence[i + 1])], bigram_counts, unigram_counts, unigram_probabilities, smooth=True)
        bigram_probability = bigram_probability[(sentence[i], sentence[i + 1])]

        if use_log:
            sentence_probability += math.log(bigram_probability, 2)
        else:
            sentence_probability *= bigram_probability

    if use_log:
        return math.pow(2, sentence_probability)
    else:
        return sentence_probability

def augment_sentence(bigram_counts, unigram_counts, unigram_probabilities, smooth, word_count):
    sentence = '<s>'
    previous_word = '<s>'
    while previous_word != '</s>':
        potential_successors = {}
        for bigram in list(bigram_counts.keys()):
            if previous_word == bigram[0]:
                successor = bigram[1]

                bigram_probability = calculate_bigram_probabilities([(previous_word, successor)], bigram_counts, unigram_counts, unigram_probabilities, smooth=True)
                bigram_probability = bigram_probability[(previous_word, successor)]
                potential_successors[successor] = bigram_probability

        divisor = sum(potential_successors.values())
        for successor in potential_successors:
            potential_successors[successor] = potential_successors[successor] / divisor

        next_word = random.choices(list(potential_successors.keys()), weights=potential_successors.values(), k=1)[0]
        sentence = sentence + ' ' + next_word
        previous_word = next_word

    if previous_word != '</s>':
        sentence = sentence + ' </s>'

    return sentence

def main():
    smooth = True
    use_log = True
    corpus_text = [
        "<s> man marries women </s>",
        "<s> woman marries man </s>",
        "<s> woman marries woman </s>",
        "<s> man divorces woman </s>",
        "<s> woman divorces man </s>"
    ]

    corpus = [re.split(r'\s+', line.strip('\n"')) for line in corpus_text]

    unigrams, bigrams, bigram_counts, unigram_counts, word_count = calculate_counts(corpus)
    unigram_probabilities = calculate_unigram_probabilities(unigrams, unigram_counts, word_count)
    bigram_probabilities = calculate_bigram_probabilities(bigrams, bigram_counts, unigram_counts, unigram_probabilities, smooth)

    # query_sentence = ["<s>", "i", "i", "am", "not", "</s>"]
    query_sentence = ["<s>", "man", "marries", "man","</s>"]

    # Print Unigram Count Table
    unigram_count_table = [["Unigram"] + query_sentence]
    unigram_count_table.append(["Count"] + [unigram_counts[word] for word in query_sentence])
    print("Unigram Count Table:")
    print(tabulate(unigram_count_table, tablefmt="grid"))

    # Print Bigram Count Table
    bigram_count_table = [[""] + query_sentence]
    for word1 in query_sentence:
        row = [word1]
        for word2 in query_sentence:
            count = bigram_counts.get((word1, word2), 0)
            row.append(count)
        bigram_count_table.append(row)
    print("\nBigram Count Table:")
    print(tabulate(bigram_count_table, headers="firstrow", tablefmt="grid"))

    sentence_probability = calculate_sentence_probability(query_sentence, unigram_counts, bigram_counts, unigram_probabilities, word_count, smooth, use_log)


 # Calculate the bigram probability table
    bigram_prob_table = [[""] + query_sentence]
    for word1 in query_sentence:
        row = [word1]
        for word2 in query_sentence:
            bigram_count = bigram_counts.get((word1, word2), 0)
            unigram_count = unigram_counts.get(word1, 0)  # Use get() to avoid KeyError
            prob = bigram_count / unigram_count if unigram_count != 0 else 0
            row.append(prob)
        bigram_prob_table.append(row)

    # Print Bigram Probability Table
    print("\nBigram Probability Table:")
    print(tabulate(bigram_prob_table, headers="firstrow", tablefmt="grid"))

    # Calculate sentence probability using diagonals of the bigram probability table
    sentence_probability = 1.0
    for i in range(len(query_sentence) - 1):
        word1, word2 = query_sentence[i], query_sentence[i + 1]
        bigram_prob = bigram_probabilities.get((word1, word2), 0)
        sentence_probability *= bigram_prob

    # print("\nSentence Probability:", sentence_probability)
    sentence_probability = calculate_sentence_probability(query_sentence, unigram_counts, bigram_counts, unigram_probabilities, word_count, smooth, use_log)

    # Print sentence probability
    print("\nSentence Probability:", sentence_probability)


if __name__ == '__main__':
    main()

Unigram Count Table:
+---------+-----+-----+---------+-----+------+
| Unigram | <s> | man | marries | man | </s> |
+---------+-----+-----+---------+-----+------+
| Count   | 5   | 4   | 3       | 4   | 5    |
+---------+-----+-----+---------+-----+------+

Bigram Count Table:
+---------+-------+-------+-----------+-------+--------+
|         |   <s> |   man |   marries |   man |   </s> |
+=========+=======+=======+===========+=======+========+
| <s>     |     0 |     2 |         0 |     2 |      0 |
+---------+-------+-------+-----------+-------+--------+
| man     |     0 |     0 |         1 |     0 |      2 |
+---------+-------+-------+-----------+-------+--------+
| marries |     0 |     1 |         0 |     1 |      0 |
+---------+-------+-------+-----------+-------+--------+
| man     |     0 |     0 |         1 |     0 |      2 |
+---------+-------+-------+-----------+-------+--------+
| </s>    |     0 |     0 |         0 |     0 |      0 |
+---------+-------+-------+-----------+

# Predicting the next word
using the same way, the most probable next word can be estimated.

In [ ]:
    corpus_text = [
        "<s> i am jack </s>",
        "<s> jack i am </s>",
        "<s> jack i like </s>",
        "<s> jack i do like </s>",
        "<s> do i like jack </s>"
    ]

    # 1. <s> jack
    # 2. <s> jack i do
    # 3. jack i am jack
    # 4. do i like